# pagos_mtm, ¿Una base?

- Al parecer alguna prueba puntual.
- Por ahí tiene que ver con esas querieas con variables y tablas virtuales declaradas

In [6]:
import os
import sys
import datetime as dtm
import pandas as pd
import pyodbc

In [7]:
#// 2. Defining Classes (Classes definition)

class MSSQLdb:
    '''MSSQLdb Class: to manage interaction w/SQLSRV databases'''
    __cnt = 0
    catalog = {'Lista de conexiones a DB NO establecidas': [],
               'Lista de Errores de conexiones a DBs': [],
               'Lista de DBs Conectadas': []}
    
    def __init__(self, srv, db, usr=None, pwd=None, port=1433):
        pyodbc.pooling = False
        MSSQLdb.__cnt += 1
        self.id = MSSQLdb.__cnt
        self.drv = '{ODBC Driver 17 for SQL Server}'
        self.srv, self.port, self.db = srv, port, db
        self.usr, self.pwd = usr, pwd
        self.id_nm = f'''{self.id} _ * {self.srv} {self.db} *'''
        if usr and pwd:
            cnxstr = f'DRIVER={self.drv};SERVER={self.srv};PORT={self.port};\
                DATABASE={self.db};UID={self.usr};PWD={self.pwd}'
        else:
            cnxstr = f"DRIVER={self.drv};SERVER={self.srv};PORT={self.port};\
                DATABASE={self.db};Trusted_Connection=yes"
        self._cnx = pyodbc.connect(cnxstr)
        self._cur = self._cnx.cursor()
        self.closed = False

    def close(self):
        self._cur.close()
        self._cnx.close()
        self.closed = True
    
    def execute(self, sql, params=None):
        self._cur.execute(sql, params or ())
    
    def qry_to_df(self, sql, params=None):
        self.execute(sql, params)
        cols = [i[0] for i in self._cur.description]
        dats = [list(xx) for xx in self._cur]
        return pd.DataFrame(data=dats, columns=cols)

In [8]:
#// 3. Global Variables (Constants)
## Datetime Vars
now = dtm.datetime.now()
thisyr, thismnth = now.year, now.month      # To validate period red from list_tasks_file
now_fname = now.strftime('%Y%m%d%H%M')      # datetime firm for filenames

lst_fn = 'CSVs_pagos_mtm1.xlsx'
lst = 'listas/' + lst_fn

#// 4. Functions Definition
## Get f_corte (last day of period): AAAAMMDD  - last day of a month
def get_last_day_date(per):
    prd_as_date = dtm.datetime.strptime(per, '%Y%m').date()
    # day 25 exists in every month. 9 days later, it's always next month
    nxt_mnth = prd_as_date.replace(day=25) + dtm.timedelta(days=9)
    # subtracting the number of days of nxt_mnth we'll get last day of original month
    last_day_date = nxt_mnth - dtm.timedelta(days=nxt_mnth.day)
    return dtm.datetime.strftime(last_day_date, '%Y%m%d')

## Get f_corte_12: fecha de corte same month last year
def get_corte_last_year(per):
    prd_as_date = dtm.datetime.strptime(per, '%Y%m').date()
    prd_12_date = prd_as_date.replace(year= prd_as_date.year - 1)
    prd_12 = dtm.datetime.strftime(prd_12_date, '%Y%m')
    return get_last_day_date(prd_12)

#// 6. Read list_tasks_file and Get date values from period
## 6.1. Read list tasks_files: CSVs_list.xlsx
try:
    lst_df = pd.read_excel(lst)
except Exception as e:
    ln = f'''>ERROR en 6.1.! Falló la lectura de {lst}
        {e}'''
    print(ln)
    # wrt_info(ln, level='err_')
    # exit_prg()
else:
    # Display little sample of the result
    cols = ['Periodo', 'Carpeta', 'Nombre archivo CSV', 'Server Base']
    ln = f'''> {lst} leído correctamente:\n{lst_df[cols].head(3)}'''
    # wrt_info(ln, level='ok_')

## 6.2. Get period, f_corte and fcorte_12 as strings
try:
    period = str(int(lst_df.loc[0, 'Periodo']))

    ## Validate period: 6 chars
    assert len(period) == 6
    peryr, permnth = int(period[:4]), int(period[4:])
    # 4 first digs: year >= 2014 and <= actual year
    assert peryr >= 2013 and peryr <= thisyr
    # last 2 digs: month >= 1 and < actual if year==thisyear else 12
    assert permnth >= 1 and permnth < (thismnth if peryr == thisyr else 13)
except Exception as e:
    ln = f'''>ERROR en 5.2.! Periodo cargado: '{period}' INVALIDO!
            Períodos válidos: de 201401 a {thisyr}{thismnth-1}
            {e} '''
    # wrt_info(ln, level='err_')
    # exit_prg()
else:
    f_corte = "'" + get_last_day_date(period) + "'"
    fcorte_12 = f"'{get_corte_last_year(period)}'"
    ln = f'''> Fecha leída, Periodo: {period}.- Calculados; Fecha de corte: {f_corte},
            Corte del mismo mes del año anterior: {fcorte_12} '''
    # wrt_info(ln, level='ok_')

## 6.3 Get the total number of Files to be generated
total_files_to_process = len(lst_df)

print(lst)
print(f_corte, fcorte_12)
lst_df

listas/CSVs_pagos_mtm1.xlsx
'20231231' '20221231'


,Periodo,Carpeta,ix,Nombre archivo CSV,Server Base,Modo de armado,Disk,Path
0,202312,LGD_tarj,1,pagos_mtm1,172.16.1.12\MZ12 MPS_Cautivas,"select cta_bto,tip_tar,nro_cta,convert(datetim...",C,MTM_PCE_00/CVS_periodo


In [9]:
#// 7. Replace period, f_corte, fcorte_12 in 'Path' and 'Modo de armado' of CVSs_list
lst_df['Path'] = lst_df['Path'].str.replace('periodo', period)
lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace('f_corte', f_corte)
lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace('fcorte_12', fcorte_12)
# replace strange "’" that appears w/libre office .xlsx
lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace("’", "'")

lst_df.loc[0, 'Modo de armado']

"select cta_bto,tip_tar,nro_cta,convert(datetime,fec_com,121) as fec_com,nro_cup,ctd_cuo,cod_mov,convert(float,sum(imp_cuo)) as imp_cuo from mps_cautivas..movtrjpro_2016_10\nwhere fec_com between '20180101' and '20231231' and cod_mov in (1,98) and tip_tar='PS' and nro_cta<=57845\ngroup by cta_bto,tip_tar,nro_cta,fec_com,nro_cup,ctd_cuo,cod_mov\norder by nro_cta,fec_com"

In [10]:
try:                        # 1. connect to de DB - db_obj
    sb = lst_df.loc[0, 'Server Base'].split()
    srv, dbnm = sb[0], sb[1]
    srv_db = f'''{srv} {dbnm}'''
    ln = f''' DB_ Conectando con: {srv_db}...'''
    # wrt_info(ln)
    db = MSSQLdb(srv, dbnm, 'usr_motor', 'MTM.m0t0R2024')
except Exception as e:
    ln = f'''>DB_ERROR en 10.a! Falló la conexión con {srv_db}
            {e}'''
    MSSQLdb.catalog['Lista de Errores de conexiones a DBs'].append(ln)
    MSSQLdb.catalog['Lista de conexiones a DB NO establecidas'].append(srv_db )
    print(ln)
        # wrt_info(ln, level='err_')
        # continue
else:
    ln = f'''> DB_ Conexión con {srv} {dbnm} ESTABLECIDA! ({db.id_nm})'''
    MSSQLdb.catalog['Lista de DBs Conectadas'].append(db.id_nm)
    print(ln) 
        # wrt_info(ln, level='ok_')
    
total_rows = len(lst_df)
ln = f'Iniciando procesamiento de queries a {srv} {dbnm} - {total_rows} queries'
print(ln)

for row in range(total_rows):       # 2. loop in e/row of mini_df
    # 2.1. create cvs_obj - prev get vars from row
    folder = lst_df.loc[row, 'Carpeta']
    fn = lst_df.loc[row, 'Nombre archivo CSV']
    qry = lst_df.loc[row, 'Modo de armado']
    disk = lst_df.loc[row, 'Disk']
    path = lst_df.loc[row, 'Path']

df = db.qry_to_df(qry)

dic = df.dtypes.to_dict()
for col, typ in dic.items():
    if typ == 'datetime64[ns]':
        df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M:%S')

> DB_ Conexión con 172.16.1.12\MZ12 MPS_Cautivas ESTABLECIDA! (1 _ * 172.16.1.12\MZ12 MPS_Cautivas *)
Iniciando procesamiento de queries a 172.16.1.12\MZ12 MPS_Cautivas - 1 queries


In [11]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 1000)
df

,cta_bto,tip_tar,nro_cta,fec_com,nro_cup,ctd_cuo,cod_mov,imp_cuo
0,101968,PS,113,2018-01-19 00:00:00,1214992,1,1,-930.00
1,101968,PS,113,2018-01-31 00:00:00,1219354,1,1,-500.00
2,101968,PS,113,2018-03-05 00:00:00,1241431,1,1,-1600.00
3,101968,PS,113,2018-04-16 00:00:00,1262821,1,1,-1099.93
4,101968,PS,113,2018-05-11 00:00:00,1277894,1,1,-2200.00
...,...,...,...,...,...,...,...,...
790906,351626,PS,57845,2018-08-13 00:00:00,1328225,1,1,-3500.00
790907,351626,PS,57845,2018-08-13 00:00:00,1328227,1,1,-0.05
790908,351626,PS,57845,2019-01-08 00:00:00,1413336,1,1,-8206.52
790909,351626,PS,57845,2019-04-10 00:00:00,1466775,1,1,-45266.35


In [12]:
df.columns
df['Aofval']
df['Aofvto']
df.info()

KeyError: 'Aofval'